# Exercise 1: Data Analysis
## Correlation Matrix Estimation and Adjustment

In [ ]:
# Import required libraries
import pandas as pd
import numpy as np
from scipy.linalg import eigh, norm
import os
import matplotlib.pyplot as plt
import seaborn as sns

# Set style
sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (10, 6)

## Define Adjustment Functions

First, we'll define the two adjustment methods from Exercise 2.

In [ ]:
def spectral_decomposition(corr_matrix, epsilon=1e-8):
    """Adjust correlation matrix using Spectral Decomposition Method."""
    if isinstance(corr_matrix, pd.DataFrame):
        corr_matrix = corr_matrix.values
    
    eigenvalues, eigenvectors = eigh(corr_matrix)
    eigenvalues_adj = np.where(eigenvalues < epsilon, epsilon, eigenvalues)
    adj_corr_matrix = eigenvectors @ np.diag(eigenvalues_adj) @ eigenvectors.T
    sqrt_diag = np.sqrt(np.diag(adj_corr_matrix))
    adj_corr_matrix = adj_corr_matrix / sqrt_diag[:, None] / sqrt_diag[None, :]
    
    return adj_corr_matrix

def alternating_projection(corr_matrix, tolerance=1e-12, tau=1e-8):
    """Adjust correlation matrix using Alternating Projection Method."""
    if isinstance(corr_matrix, pd.DataFrame):
        corr_matrix = corr_matrix.values
    
    Y = corr_matrix.copy()
    delta_S = 0
    k = 1
    b = np.ones(corr_matrix.shape[0])
    
    while True:
        R = Y - delta_S
        eigenvals_R, eigenvecs_R = eigh(R)
        eigenvals_R[eigenvals_R < tau] = tau
        X_k = eigenvecs_R @ np.diag(eigenvals_R) @ eigenvecs_R.T
        delta_S = X_k - R
        Y = X_k - np.diag(X_k.diagonal() - b)
        norm_k = norm(X_k.diagonal() - b)
        
        if norm_k <= tolerance:
            break
        else:
            k += 1
            
        if k > 10000:
            print(f"Warning: Maximum iterations reached. Current norm: {norm_k}")
            break
    
    X_k_scalar = np.diag(1/np.sqrt(np.diag(X_k)))
    adj_corr_matrix = X_k_scalar @ X_k @ X_k_scalar
    distance = np.linalg.norm(corr_matrix - adj_corr_matrix)
    
    return adj_corr_matrix, k, distance

## a) Read the Data Files

In [ ]:
# Determine the correct path to data files
if os.path.exists('../Data_HW-20251128/set_1.xlsx'):
    data_path = '../Data_HW-20251128/'
elif os.path.exists('Data_HW-20251128/set_1.xlsx'):
    data_path = 'Data_HW-20251128/'
else:
    data_path = '../Data_HW-20251128/'

# Read the Excel files
set_1_raw = pd.read_excel(data_path + 'set_1.xlsx')
set_2_raw = pd.read_excel(data_path + 'set_2.xlsx')
set_3_raw = pd.read_excel(data_path + 'set_3.xlsx')

print("Data files loaded successfully!")
print(f"\nSet 1 - Shape: {set_1_raw.shape}")
print(f"Columns: {list(set_1_raw.columns)}")
print(f"\nFirst few rows of Set 1:")
print(set_1_raw.head())

In [ ]:
# Select only numeric columns (exclude date columns)
set_1 = set_1_raw.select_dtypes(include=[np.number])
set_2 = set_2_raw.select_dtypes(include=[np.number])
set_3 = set_3_raw.select_dtypes(include=[np.number])

print("Numeric columns selected:")
print(f"\nSet 1 - Shape: {set_1.shape}, Columns: {list(set_1.columns)}")
print(f"Set 2 - Shape: {set_2.shape}, Columns: {list(set_2.columns)}")
print(f"Set 3 - Shape: {set_3.shape}, Columns: {list(set_3.columns)}")

## b) Calculate Correlation Matrices

In [ ]:
# Calculate correlation matrices
corr_1 = set_1.corr()
corr_2 = set_2.corr()
corr_3 = set_3.corr()

print("="*70)
print("CORRELATION MATRIX - SET 1")
print("="*70)
print(corr_1)

print("\n" + "="*70)
print("CORRELATION MATRIX - SET 2")
print("="*70)
print(corr_2)

print("\n" + "="*70)
print("CORRELATION MATRIX - SET 3")
print("="*70)
print(corr_3)

## c) Check if Matrices Need Adjustment

A correlation matrix needs adjustment if it is **not positive semi-definite**, which occurs when one or more eigenvalues are negative or zero.

In [ ]:
def check_positive_definite(corr_matrix, name):
    """
    Check if a correlation matrix is positive definite.
    A matrix is positive definite if all eigenvalues are positive.
    """
    eigenvalues, _ = eigh(corr_matrix)
    
    print(f"\n{'='*70}")
    print(f"ANALYSIS: {name}")
    print(f"{'='*70}")
    print(f"Eigenvalues: {eigenvalues}")
    print(f"Minimum eigenvalue: {eigenvalues.min():.10f}")
    print(f"Maximum eigenvalue: {eigenvalues.max():.10f}")
    
    if eigenvalues.min() > 0:
        print(f"\n✓ Matrix is POSITIVE DEFINITE")
        print(f"  All eigenvalues are positive.")
        print(f"  NO ADJUSTMENT NEEDED.")
        return True
    else:
        print(f"\n✗ Matrix is NOT POSITIVE DEFINITE")
        print(f"  Found {(eigenvalues <= 0).sum()} non-positive eigenvalue(s).")
        print(f"  ADJUSTMENT IS REQUIRED.")
        return False

# Check all three matrices
is_pd_1 = check_positive_definite(corr_1, "SET 1")
is_pd_2 = check_positive_definite(corr_2, "SET 2")
is_pd_3 = check_positive_definite(corr_3, "SET 3")

needs_adjustment_1 = not is_pd_1
needs_adjustment_2 = not is_pd_2
needs_adjustment_3 = not is_pd_3

### Justification

**Why does a correlation matrix need adjustment?**

A valid correlation matrix must be **positive semi-definite**, meaning all eigenvalues ≥ 0. When this property is violated:

1. **Mathematical invalidity**: The matrix doesn't represent a valid covariance structure
2. **Computational issues**: Cannot perform Cholesky decomposition, needed for simulations
3. **Statistical problems**: Cannot be used in many multivariate analyses

**Common causes:**
- **Missing data**: Pairwise correlation calculations with different sample sizes
- **Small sample size**: Estimation errors accumulate
- **Numerical precision**: Rounding errors in computation
- **Data quality**: Inconsistent or contradictory correlation estimates

**Decision rule:**
- If minimum eigenvalue < 0: **Definitely needs adjustment**
- If minimum eigenvalue ≈ 0 (e.g., < 1e-8): **May need adjustment** depending on application
- If all eigenvalues > 0: **No adjustment needed**

## Apply Adjustments to Matrices that Need It

In [ ]:
def adjust_and_compare(corr_matrix, name):
    """
    Apply both adjustment methods and compare results.
    """
    print(f"\n{'='*70}")
    print(f"ADJUSTING {name}")
    print(f"{'='*70}")
    
    # Apply Spectral Decomposition
    print("\n[1] Spectral Decomposition Method")
    print("-" * 70)
    corr_spectral = spectral_decomposition(corr_matrix)
    eigenvals_spectral, _ = eigh(corr_spectral)
    distance_spectral = np.linalg.norm(corr_matrix - corr_spectral)
    
    print(f"Adjusted eigenvalues: {eigenvals_spectral}")
    print(f"Minimum eigenvalue:   {eigenvals_spectral.min():.10f}")
    print(f"Distance from original: {distance_spectral:.10f}")
    
    # Apply Alternating Projection
    print("\n[2] Alternating Projection Method")
    print("-" * 70)
    corr_alt, iterations, distance_alt = alternating_projection(corr_matrix)
    eigenvals_alt, _ = eigh(corr_alt)
    
    print(f"Number of iterations: {iterations}")
    print(f"Adjusted eigenvalues: {eigenvals_alt}")
    print(f"Minimum eigenvalue:   {eigenvals_alt.min():.10f}")
    print(f"Distance from original: {distance_alt:.10f}")
    
    # Comparison
    print("\n[3] Comparison")
    print("-" * 70)
    print(f"Spectral Decomposition distance: {distance_spectral:.10f}")
    print(f"Alternating Projection distance:  {distance_alt:.10f}")
    print(f"Difference: {abs(distance_spectral - distance_alt):.10f}")
    print(f"\nBetter method (smaller distance): ", end="")
    if distance_spectral < distance_alt:
        print("Spectral Decomposition")
    else:
        print("Alternating Projection")
    
    return corr_spectral, corr_alt

# Store adjusted matrices
adjusted_matrices = {}

if needs_adjustment_1:
    corr_1_spectral, corr_1_alt = adjust_and_compare(corr_1, "SET 1")
    adjusted_matrices['set_1'] = {'spectral': corr_1_spectral, 'alternating': corr_1_alt}

if needs_adjustment_2:
    corr_2_spectral, corr_2_alt = adjust_and_compare(corr_2, "SET 2")
    adjusted_matrices['set_2'] = {'spectral': corr_2_spectral, 'alternating': corr_2_alt}

if needs_adjustment_3:
    corr_3_spectral, corr_3_alt = adjust_and_compare(corr_3, "SET 3")
    adjusted_matrices['set_3'] = {'spectral': corr_3_spectral, 'alternating': corr_3_alt}

## Visualizations

In [ ]:
# 1. Eigenvalue comparison for all three datasets
fig, axes = plt.subplots(1, 3, figsize=(15, 4))

datasets = [('Set 1', corr_1), ('Set 2', corr_2), ('Set 3', corr_3)]
needs_adj = [needs_adjustment_1, needs_adjustment_2, needs_adjustment_3]

for ax, (name, corr), needs in zip(axes, datasets, needs_adj):
    eigenvals, _ = eigh(corr)
    colors = ['red' if ev <= 0 else 'green' for ev in eigenvals]
    ax.bar(range(len(eigenvals)), eigenvals, color=colors, alpha=0.7, edgecolor='black')
    ax.axhline(y=0, color='black', linestyle='--', linewidth=2)
    ax.set_xlabel('Eigenvalue Index', fontsize=10)
    ax.set_ylabel('Eigenvalue', fontsize=10)
    ax.set_title(f'{name}\n{"✗ Needs Adjustment" if needs else "✓ Valid"}', fontsize=11)
    ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.suptitle('Eigenvalue Analysis - All Datasets', y=1.02, fontsize=14, fontweight='bold')
plt.show()

In [ ]:
# 2. Correlation matrix heatmaps
fig, axes = plt.subplots(1, 3, figsize=(15, 4))

for ax, (name, corr) in zip(axes, datasets):
    sns.heatmap(corr, annot=True, fmt='.2f', cmap='RdBu_r', center=0,
                vmin=-1, vmax=1, square=True, ax=ax, cbar_kws={'shrink': 0.8})
    ax.set_title(name, fontsize=12)

plt.tight_layout()
plt.suptitle('Original Correlation Matrices', y=1.02, fontsize=14, fontweight='bold')
plt.show()

In [ ]:
# 3. Adjustment comparison (only for matrices that need adjustment)
if needs_adjustment_1 or needs_adjustment_3:
    # Count how many need adjustment
    n_to_adjust = sum([needs_adjustment_1, needs_adjustment_3])
    
    fig, axes = plt.subplots(1, n_to_adjust, figsize=(7*n_to_adjust, 5))
    if n_to_adjust == 1:
        axes = [axes]
    
    idx = 0
    for i, (needs, name) in enumerate([(needs_adjustment_1, 'Set 1'), 
                                         (needs_adjustment_3, 'Set 3')]):
        if needs:
            corr = corr_1 if i == 0 else corr_3
            
            # Get eigenvalues
            eigenvals_orig, _ = eigh(corr)
            corr_spec = spectral_decomposition(corr)
            eigenvals_spec, _ = eigh(corr_spec)
            corr_alt, _, _ = alternating_projection(corr)
            eigenvals_alt, _ = eigh(corr_alt)
            
            # Plot
            x = np.arange(len(eigenvals_orig))
            width = 0.25
            
            axes[idx].bar(x - width, eigenvals_orig, width, label='Original', 
                         color='red', alpha=0.7)
            axes[idx].bar(x, eigenvals_spec, width, label='Spectral', 
                         color='blue', alpha=0.7)
            axes[idx].bar(x + width, eigenvals_alt, width, label='Alternating', 
                         color='green', alpha=0.7)
            
            axes[idx].axhline(y=0, color='black', linestyle='--', linewidth=1)
            axes[idx].set_xlabel('Eigenvalue Index', fontsize=11)
            axes[idx].set_ylabel('Eigenvalue', fontsize=11)
            axes[idx].set_title(f'{name} - Adjustment Comparison', fontsize=12, fontweight='bold')
            axes[idx].legend()
            axes[idx].grid(True, alpha=0.3)
            
            idx += 1
    
    plt.tight_layout()
    plt.show()
else:
    print("No matrices needed adjustment for visualization.")

## Summary of Results

In [ ]:
print("\n" + "="*70)
print("FINAL SUMMARY")
print("="*70)

print(f"\nSet 1: {'✗ Needs adjustment' if needs_adjustment_1 else '✓ No adjustment needed'}")
if needs_adjustment_1:
    eigenvals_orig = eigh(corr_1)[0]
    print(f"  Original min eigenvalue: {eigenvals_orig.min():.10f}")
    print(f"  Reason: Negative eigenvalue (likely due to missing data)")

print(f"\nSet 2: {'✗ Needs adjustment' if needs_adjustment_2 else '✓ No adjustment needed'}")
if not needs_adjustment_2:
    eigenvals_orig = eigh(corr_2)[0]
    print(f"  Original min eigenvalue: {eigenvals_orig.min():.10f}")
    print(f"  Reason: All eigenvalues positive (complete data, valid correlations)")

print(f"\nSet 3: {'✗ Needs adjustment' if needs_adjustment_3 else '✓ No adjustment needed'}")
if needs_adjustment_3:
    eigenvals_orig = eigh(corr_3)[0]
    print(f"  Original min eigenvalue: {eigenvals_orig.min():.10e}")
    print(f"  Reason: Near-zero eigenvalue (perfect/near-perfect correlations)")

print("\n" + "-"*70)
print("CONCLUSIONS:")
print("-"*70)
print("\n1. Both adjustment methods successfully produce positive semi-definite matrices")
print("2. Alternating Projection typically provides closer approximation (smaller distance)")
print("3. Spectral Decomposition is faster and simpler to implement")
print("4. Choice of method depends on application requirements")